In [2]:
import prefect as pf
import prefect.blocks.system as pfbs
import time


In [3]:
import polars as pl

In [4]:
from polygon import RESTClient
from urllib3.exceptions import MaxRetryError

In [5]:
polygon_key_block = await pfbs.Secret.load("polygon-api-key")
polygon_key = polygon_key_block.get()

In [6]:
plg = RESTClient(polygon_key)

In [7]:
import time

In [8]:

ticker_lister = plg.list_tickers(
    market="stocks",
    type="CS",
    active="true",
    limit=1000,
    # params={"ticker.gt": tickers[-1].ticker}
)


In [9]:

def iterate_ticker_lister(lister):
    start_time = time.monotonic()
    while True:
        try:
            yield next(lister)
        except MaxRetryError:
            print(f"Sleeping for {60 - (time.monotonic() - start_time)}")
            time.sleep(60 - (time.monotonic() - start_time))
            start_time = time.monotonic()


tickers = []
for ticker in iterate_ticker_lister(ticker_lister):
    tickers.append(ticker)
    if len(tickers) % 500 == 0:
        print(len(tickers))


500
1000
1500
2000
2500
3000
3500
4000
4500
5000
Sleeping for 57.438298995024525


RuntimeError: generator raised StopIteration

In [12]:
ticker_lister = plg.list_tickers(
    market="stocks",
    type="CS",
    active="true",
    limit=1000,
    params={"ticker.gt": tickers[-1].ticker}
)

for ticker in iterate_ticker_lister(ticker_lister):
    tickers.append(ticker)
    if len(tickers) % 500 == 0:
        print(len(tickers))

RuntimeError: generator raised StopIteration

In [14]:
len(tickers)

5314

In [15]:
import datetime

In [16]:
from enum import Enum
import datetime
class BronzeConstants(Enum):
    TIMESTAMP_INSERTION_COLUMN = "ingestion_timestamp"

In [18]:
tickers_pl = (pl.DataFrame(tickers)
        .with_columns(
            pl.lit(datetime.datetime.utcnow()).alias(BronzeConstants.TIMESTAMP_INSERTION_COLUMN.value),
        ))

In [19]:
import os
os.environ["AWS_S3_ALLOW_UNSAFE_RENAME"] = "true"

In [22]:
from prefect_etl.storage_config import MinIOCredentials
from flows.polygon_elt.get_aggregates import MINIO_CREDENTIAL_SECRET_KEY
minio_credentials = await MinIOCredentials.load(MINIO_CREDENTIAL_SECRET_KEY)

In [24]:
minio_credentials.rust_s3_kwargs

{'endpoint': 'http://localhost:9000',
 'access_key_id': 'c4dqtQY3jEc1sp7ErJf4',
 'secret_access_key': 'V0JTYpL2pv4RDlShahWIzNCuttxbVVKMea1OWBoI',
 'region': 'us-east-1',
 'allow_http': 'true'}

In [25]:
tickers_pl.write_delta(
    f"s3://etl/polygon/bronze/tickers/",
    mode="append",
    storage_options=minio_credentials.rust_s3_kwargs,
)

In [27]:
pl.read_delta(f"s3://etl/polygon/bronze/tickers/", storage_options=minio_credentials.rust_s3_kwargs)

active,cik,composite_figi,currency_name,currency_symbol,base_currency_symbol,base_currency_name,delisted_utc,last_updated_utc,locale,market,name,primary_exchange,share_class_figi,ticker,type,source_feed,ingestion_timestamp
bool,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,datetime[μs]
true,"""0001090872""","""BBG000C2V3D6""","""usd""",null,null,null,null,"""2024-03-01T00:…","""us""","""stocks""","""Agilent Techno…","""XNYS""","""BBG001SCTQY4""","""A""","""CS""",null,2024-03-03 10:40:49.819465
true,"""0001675149""","""BBG00B3T3HD3""","""usd""",null,null,null,null,"""2024-03-01T00:…","""us""","""stocks""","""Alcoa Corporat…","""XNYS""","""BBG00B3T3HF1""","""AA""","""CS""",null,2024-03-03 10:40:49.819465
true,"""0001844817""","""BBG011XR7306""","""usd""",null,null,null,null,"""2024-03-01T00:…","""us""","""stocks""","""Armada Acquisi…","""XNAS""","""BBG011XR7315""","""AACI""","""CS""",null,2024-03-03 10:40:49.819465
true,"""0001853138""",null,"""usd""",null,null,null,null,"""2024-03-01T00:…","""us""","""stocks""","""Ares Acquisiti…","""XNYS""",null,"""AACT""","""CS""",null,2024-03-03 10:40:49.819465
true,"""0001422142""","""BBG002WN7DT2""","""usd""",null,null,null,null,"""2024-03-01T00:…","""us""","""stocks""","""Aadi Bioscienc…","""XNAS""","""BBG002WN7DV9""","""AADI""","""CS""",null,2024-03-03 10:40:49.819465
true,"""0001848898""","""BBG00ZKGK0R2""","""usd""",null,null,null,null,"""2024-03-01T00:…","""us""","""stocks""","""African Agricu…","""XNAS""","""BBG00ZKGK109""","""AAGR""","""CS""",null,2024-03-03 10:40:49.819465
true,"""0000006201""","""BBG005P7Q881""","""usd""",null,null,null,null,"""2024-03-01T00:…","""us""","""stocks""","""American Airli…","""XNAS""","""BBG005P7Q907""","""AAL""","""CS""",null,2024-03-03 10:40:49.819465
true,"""0001555074""","""BBG003PNL136""","""usd""",null,null,null,null,"""2024-03-01T00:…","""us""","""stocks""","""Altisource Ass…","""XASE""","""BBG003PNL145""","""AAMC""","""CS""",null,2024-03-03 10:40:49.819465
true,"""0000008177""","""BBG000B9XB24""","""usd""",null,null,null,null,"""2024-03-01T00:…","""us""","""stocks""","""Atlantic Ameri…","""XNAS""","""BBG001S5N8T1""","""AAME""","""CS""",null,2024-03-03 10:40:49.819465
